# 1. Анализ текстов с использованием метода Word2vec

Word2vec - это еще один агоритм преобразования текстов в точки в векторном пространстве признаков. В его основе лежит нейронная сеть - автоэнкодер, который преобразует слово в вектор фиксированной длины. Особенность этого алгоритма заключается в том, что он учитывает семантику слов. Близкие по смыслу слова будут располагаться ближе друг к другу в векторном пространстве, а далекие - дальше. Если объем текстов достаточно большой, то с помощью модели word2vec мы можем определять синонимы и антонимы слов, и, благодаря этому, точность классификации текстов может увеличиться.

Сегодня будем пользоваться теми же наборами текстов, что и в прошлый раз. Прежде всего установим уже известные вам библиотеки для работы с текстовыми данными.


In [4]:
!pip install pymorphy2 nltk
import nltk
from nltk.tokenize import word_tokenize
import nltk

import ssl

try:

    _create_unverified_https_context = ssl._create_unverified_context

except AttributeError:

    pass

else:

    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")


DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
  Using cached nltk-3.5.zip (1.4 MB)
  Using cached regex-2022.1.18.tar.gz (382 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached importlib_resources-3.3.1-py2.py3-none-any.whl (26 kB)
  Using cached typing-3.10.0.0-py2-none-any.whl (26 kB)
Using legacy 'setup.py install' for nltk, since package 'wheel' is not installed.
Using legacy 'setup.py install' for regex, since package 'wheel' is not installed.
    Running setup.py install for regex ... error
    ERROR: Command errored out with exit status 1:
     command: /Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.a

[nltk_data] Downloading package punkt to /Users/Int/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Сейчас мы готовы к проведению предобработки текста. Но для того, чтобы работать с моделью word2vec, нам еще понадобится установить библиотеку gensim, в которой описана эта модель.

In [28]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Для демонстрации возможностей этой библиотеки снова возьмем набор данных с текстами твитов, где сразу же переименуем класс -1 в 0. Вспомним, как выглядит наш датасет.

In [15]:
# from google.colab import drive
# import os
# # drive.mount('/content/drive/')
# drive.mount("/content/drive/", force_remount=True)

In [22]:
import pandas as pd

df = pd.read_excel('/Users/Int/DS/day6/data-samples/tweets_example.xlsx')
df.positive[df.positive==-1] = 0
df.loc[16:25]

/var/folders/4w/kq3dzwvx4qjg1s87_r4kd7bw0000gp/T/ipykernel_13589/1335578510.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.positive[df.positive==-1] = 0


,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
16,4.089069e+17,1.386326e+09,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1.0,0.0,1.0,0.0,2074.0,82.0,44.0,5.0
17,4.089069e+17,1.386326e+09,Tienn_En,"@marinaysol а, а то подумала, что у тебя там п...",1.0,0.0,0.0,0.0,6362.0,30.0,28.0,1.0
18,4.089069e+17,1.386326e+09,bstrd666,@xLesherx @4EU3 зря вы с этой ерундой шутите))...,1.0,0.0,0.0,0.0,13431.0,473.0,111.0,2.0
19,4.089069e+17,1.386326e+09,esken_h,@Moscow_advokat Очень главное спасибо for МЕ...,1.0,0.0,0.0,0.0,126795.0,581.0,86.0,10.0
20,4.089069e+17,1.386326e+09,Obrazcova98,"У нас есть прекрасная история, как сдохнуть за...",1.0,0.0,0.0,0.0,492.0,14.0,23.0,0.0
21,4.089068e+17,1.386326e+09,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,0.0,0.0,0.0,0.0,8064.0,111.0,94.0,2.0
22,4.089068e+17,1.386326e+09,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",0.0,0.0,0.0,0.0,26.0,42.0,39.0,0.0
23,4.089069e+17,1.386326e+09,4post21,@elina_4post как говорят обещаного три года жд...,0.0,0.0,0.0,0.0,718.0,49.0,249.0,0.0
24,4.089069e+17,1.386326e+09,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",0.0,0.0,0.0,0.0,10628.0,207.0,200.0,0.0
25,4.089069e+17,1.386326e+09,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",0.0,0.0,0.0,0.0,35.0,17.0,34.0,0.0


Так же, как и при использовании изученных раньше алгоритмов векторизации текста, нам нужно провести очистку данных, т.е. привести все слова текстов к одинаковому виду и форме. Напишем для этого функцию, чтобы можно было потом все эти действия вызывать одной командой.

In [29]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2


morph = pymorphy2.MorphAnalyzer()


def text_preprocessing(text):
    """Функция принимает строку и возвращает список слов в начальной форме"""
    text = text.lower()                                                         # приводим текст к нижнему регистру
    text = re.sub(r"[^А-Яа-я]", " ", text)                                      # удаляем все некириллические символы
    words = word_tokenize(text)                                                 # разбиваем тексты на списки слов
    words = [morph.parse(word)[0].normal_form for word in words]                # приводим слова к начальной форме
    words = [word for word in words if word not in stopwords.words("russian")]  # удаляем слова из стоп-листа
    return words

In [30]:
preprocessed_df = df[["text", "positive"]]
preprocessed_df.text = df.text.apply(text_preprocessing)
preprocessed_df[15:25]

/var/folders/4w/kq3dzwvx4qjg1s87_r4kd7bw0000gp/T/ipykernel_13589/2111668481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df.text = df.text.apply(text_preprocessing)


,text,positive
15,"[смотреть, случайно, аспирантура, попасть, нау...",1.0
16,"[эхх, угата, контрольный, предупреждать, контр...",1.0
17,"[подумать, пробежечка]",1.0
18,"[зря, ерунда, шутить, история, наблюдать, режи...",1.0
19,"[очень, главное, спасибо, медведа, работать, к...",1.0
20,"[прекрасный, история, сдохнуть, неделя]",1.0
21,"[работа, полный, пиддес, каждый, закрытие, мес...",0.0
22,"[коллега, сидеть, рубиться, долбать, винд, мочь]",0.0
23,"[говорить, обещаной, год, ждать]",0.0
24,"[желать, хороший, пол, удачный, посадка, очень...",0.0


Обратите внимание, на этот раз мы не удаляем редкие слова, поскольку теперь текст характеризуется не частотностью тех или иных слов, а семантикой слов, входящих в данный текст.

Теперь обучим модель word2vec. Поскольку это не предиктивная модель, будем обучать ее на всём множестве текстов, чтобы получить векторное представление как можно большего количества слов. Подробно о параметрах модели можно почитать в официальной [документации](https://radimrehurek.com/gensim/models/word2vec.html).

In [57]:
from gensim.models import Word2Vec

w2v = Word2Vec(vector_size=300, min_count=1)  # создадим экземпляр модели word2vec. Здесь size - размер векторного пространства,
                                       # min_count - минимальное количество появлений слова в наборе данных, при котором
                                       # будем учитывать это слово в модели
w2v.build_vocab(preprocessed_df.text)  # обучим модель на нашем наборе текстов

Теперь посмотрим, какую информацию о словах мы можем получить из обученной модели. Например, для выбранного нами слова можем посмотреть список наиболее похожих на него слов с точки зрения модели.

In [56]:
w2v.wv.most_similar(positive="ерунда")  # этот метод возвращает список кортежей, где первый элемент - это слово,
                                        # а второй - степень схожести со словом "ерунда". Чем ближе это число к 1,
                                        # тем ближе по смыслу выведенное слово

[('новый', 0.19879139959812164),
 ('вообще', 0.19217325747013092),
 ('мальчик', 0.1701221913099289),
 ('рубиться', 0.14808665215969086),
 ('рада', 0.14677517116069794),
 ('обещаной', 0.12424548715353012),
 ('винд', 0.11481524258852005),
 ('россия', 0.11323853582143784),
 ('увидеть', 0.11053305119276047),
 ('деревня', 0.10091958194971085)]

Аналогично можем посмотреть наименее похожие слова:

In [47]:
w2v.wv.most_similar(negative=["ерунда"])  # аргумент negative говорит о том, что нужно искать наименее похожие слова.
                                          # В этом случае числа - это степень непохожести.
                                          # Чем ближе к 1, тем слово меньше похоже на "ерунду"

[('общество', 0.14702370762825012),
 ('страдать', 0.14572249352931976),
 ('самый', 0.1357945054769516),
 ('задумать', 0.13516545295715332),
 ('тп', 0.13276904821395874),
 ('зелёный', 0.12756392359733582),
 ('полный', 0.11560168117284775),
 ('мск', 0.11394201219081879),
 ('шаблон', 0.11300007998943329),
 ('таки', 0.11047962307929993)]

Метод `most_similar` может помочь нам определить слова, которые наиболее похожи на один набор слов и наименее похожи на другой набор слов.

In [55]:
w2v.wv.most_similar(positive=["подняться", "угата"], negative=["погибать", "клоун"])

# на этот раз цифры - это некая общая метрика похожести на то, что мы просим

[('общество', 0.17890751361846924),
 ('гордый', 0.15167193114757538),
 ('долго', 0.14699722826480865),
 ('добрый', 0.13322556018829346),
 ('мочь', 0.1329142153263092),
 ('просто', 0.1327962726354599),
 ('ждать', 0.12339630722999573),
 ('твой', 0.1222352609038353),
 ('который', 0.10499397665262222),
 ('часть', 0.09969993680715561)]

Еще можно посмотреть, насколько похожи два слова из выборки

In [53]:
w2v.wv.similarity("клоун", "работа")  # ответ может быть отрицательным - это будет означать, что
                                         # эта пара слов - больше антонимы, чем синонимы

0.08130691

Очевидно, что с теми словами, которых не было в обучающей выборке, модель работать не сможет:

In [50]:
w2v.wv.most_similar("сбербанк")

KeyError: "Key 'сбербанк' not present"

А теперь давайте посмотрим на график, какие слова как расположены друг относительно друга. По умолчанию модель word2vec отображает все слова в пространство размерности 300. Это означает, что каждое слово превращается в набор из 300 чисел. На мониторе такую размерность отобразить очень сложно, поэтому воспользуемся методом снижения размерности векторного пространства t-SNE. Сожмем наши вектора до размерности 2, чтобы их легко можно было отобразить на плоскости.

In [1]:
from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)


def reduce_dimensions(w2v_model):
    """Фукнция принимает модель word2vec и возвращает массив абсцисс,
    массив ординат и массив слов после снижения размерности"""
    tsne = TSNE(n_components=2, random_state=256)  # создадим экземпляр модели TSNE
    vectors = np.asarray(w2v_model.wv.vectors)     # возьмем из модели 300-мерный массив слов-векторов
    labels = np.asarray(w2v_model.wv.index_to_key)   # отдельно сохраним соответствие номера вектора и самого слова
    vectors = tsne.fit_transform(vectors)          # проведем преобразование каждого вектора в 2-мерный

    x = [v[0] for v in vectors]                    # запишем отдельно массив абсцисс и массив ординат
    y = [v[1] for v in vectors]
    return x, y, labels


def plot_w2v(w2v_model):
    """Функция строит график распределения слов по векторному пространству
    размерности 2 исходя из обученной модели word2vec"""
    x, y, labels = reduce_dimensions(w2v_model)                      # получим значения по осям и названия точек (исходные слова)
    plt.scatter(x, y)                                                # строим график с точками
    words_to_show_indices = np.random.randint(len(labels), size=25)  # выберем 25 случайных слов, которые отобразим на графике
    for i in words_to_show_indices:
        plt.annotate(labels[i], (x[i], y[i]))                        # для каждого из этих 25 слов отобразим текст на картинке


plot_w2v(w2v)                                                        # применим написанные функции к обученной модели

NameError: name 'w2v' is not defined

Кроме вышеперечисленных возможностей, можно обучить модель word2vec на предсказание следующих слов. Обучим ее и попробуем предсказать продолжение твита "Котёнка вчера носик разбила, плакала и расстраивалась :("

In [62]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10000)
w2v.predict_output_word(["вчера", "носик", "разбить"])

[('плакать', 0.34837845),
 ('нка', 0.26212084),
 ('расстраиваться', 0.22428073),
 ('кот', 0.14254387),
 ('сердце', 0.0015545582),
 ('приехать', 0.0013829062),
 ('наш', 0.0010170881),
 ('котэнить', 0.00073868985),
 ('деревня', 0.00071413646),
 ('испугаться', 0.0005688586)]

Вообще говоря, списки синонимов и антонимов у нас получились достаточно спорные. Причина этого - малая выборка слов, на которых обучалась модель. Внутри word2vec используется нейросеть-автоэнкодер, а таким моделям всегда нужно много данных для того, чтобы составить корректное отображение входных данных в векторное пространство. Чем больше слов в тексте, тем понятней, какие слова похожи по значению, а какие наоборот противоположны.

# Задание 1
1. Проведите предобработку текстов из файлов positive.csv, negative.csv. Нужно выполнить те же действия, что в предыдущем дне, но не удалять редко встречающиеся слова. Регулировать использование редких слов будем на уровне модели word2vec. Не забудьте удалить стоп-слова.
2. Будем исследовать то, как влияют на качество преобразования *размер целевого векторного пространства* и *использование редких слов*. Создайте несколько моделей word2vec, перебрав параметры:
  - размер результирующего пространства: [10, 300, 500] при фиксированной минимальной встречаемости слов = 10
  - минимальная встречаемость слов: [1, 10, 100] при фиксированном размере результирующего векторного пространства = 300

  Обучите их на всем пространстве текстов.

3. Отберите 5 случайных слов из выборки позитивных публикаций и 5 случайных слови из выборки негативных публикаций.
4. Для каждой из обученных моделей найдите по 15 синонимов и по 15 антонимов для каждого из слов из п.3. Опишите:
  - как влияет размер результирующего пространства на точность определения синонимов/антонимов моделью? почему?
  - как влияет минимальная встречаемость слов на точность определения синонимов/антонимов моделью? почему?
5. Постройте графики распределения слов в двумерном пространстве. Опишите, как влияют исследуемые параметры на кучность и расположение точек на графике. Почему?
6. Возьмите любой твит, обучите модель word2vec с параметрами по умолчанию и попробуйте предсказать продолжение твита. Также попробуйте предсказать продолжение случайной фразы. Сравните результаты, полученные после обучения моделей с разным количеством эпох обучения.

In [3]:
# код выполнения задания 1
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

df = pd.read_excel('/Users/Int/DS/day6/data-samples/tweets_example.xlsx')
df.loc[16:25]
columns = df.columns
pos = pd.read_csv('/Users/Int/DS/day6/datasets/positive.csv', sep=';', names=columns, error_bad_lines=False)
neg = pd.read_csv('/Users/Int/DS/day6/datasets/negative.csv',  sep=';', names=columns, error_bad_lines=False)

df = pd.concat([pos, neg],ignore_index=True)

if (df['id'].is_unique):
	print("[Ok] Все id уникальны.")
else:
	print("[Ошибка!] Повторяющиеся id!")
print(df.shape)


df.index.is_monotonic_increasing

df.text = df.text.str.lower()

df.text = df.text.str.replace(r"[^А-Яа-я]"," ")

df.text = list(map(word_tokenize, df.text))
df.text.loc[19:22]
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
preprocessed_df = df[["text", "positive"]]
preprocessed_df
# preprocessed_df = df

from gensim.models import Word2Vec
#2
# w2v10 = Word2Vec(vector_size=10, min_count=10)
# w2v300 = Word2Vec(vector_size=300, min_count=10)
# w2v500 = Word2Vec(vector_size=500, min_count=10)

w2v1 = Word2Vec(vector_size=300, min_count=1)  # создадим экземпляр модели word2vec. Здесь size - размер векторного пространства,
                                       # min_count - минимальное количество появлений слова в наборе данных, при котором
                                      # будем учитывать это слово в модели
# w2v10 = Word2Vec(vector_size=300, min_count=10)
# w2v100 = Word2Vec(vector_size=300, min_count=100)

w2v.build_vocab(preprocessed_df.text)

from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)


def reduce_dimensions(w2v_model):
    """Фукнция принимает модель word2vec и возвращает массив абсцисс,
    массив ординат и массив слов после снижения размерности"""
    tsne = TSNE(n_components=2, random_state=256)  # создадим экземпляр модели TSNE
    vectors = np.asarray(w2v_model.wv.vectors)     # возьмем из модели 300-мерный массив слов-векторов
    labels = np.asarray(w2v_model.wv.index_to_key)   # отдельно сохраним соответствие номера вектора и самого слова
    vectors = tsne.fit_transform(vectors)          # проведем преобразование каждого вектора в 2-мерный

    x = [v[0] for v in vectors]                    # запишем отдельно массив абсцисс и массив ординат
    y = [v[1] for v in vectors]
    return x, y, labels


def plot_w2v(w2v_model):
    """Функция строит график распределения слов по векторному пространству
    размерности 2 исходя из обученной модели word2vec"""
    x, y, labels = reduce_dimensions(w2v_model)                      # получим значения по осям и названия точек (исходные слова)
    plt.scatter(x, y)                                                # строим график с точками
    words_to_show_indices = np.random.randint(len(labels), size=25)  # выберем 25 случайных слов, которые отобразим на графике
    for i in words_to_show_indices:
        plt.annotate(labels[i], (x[i], y[i]))                        # для каждого из этих 25 слов отобразим текст на картинке


plot_w2v(w2v) 
#3
# words_to_show_pos = np.random.randint(df.positive[df.positive==-1] , size=5) 
# words_to_show_neg = np.random.randint(df.positive[df.positive==1] , size=5) 
# words_to_show_pos
# words_to_show_neg
# w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10000)
# w2v.predict_output_word(["вчера", "носик", "разбить"])

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
[Ok] Все id уникальны.
(226834, 12)


NameError: name 'word_tokenize' is not defined

In [99]:
# 4 Выберем случайные слова из датафрейма
import random

set = []
preprocessed_df = preprocessed_df[df.positive == 1].text
# Выберем пять позитивных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df))
  word = random.sample(str_p, 1)
  set.append(word)

# Выберем пять негативных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df[df.positive == 0].text))
  word = random.sample(str_p, 1)
  set.append(word)

set

AttributeError: 'Series' object has no attribute 'text'

# 2. Использование градиентного бустинга над решающими деревьями для решения задачи классификации текстов

В предыдущем разделе мы научились превращать слова в векторы. Но перед нами стоит задача классификации текста, а не одного слова, поэтому нам нужно придумать способ, как целый текст представить числами.

После преобразования отдельные слова стали векторами, значения которых зависят от семантики слова. Будем рассматривать твит как сущность с усредненной семантикой всех содержащихся в нём слов. Таким образом, для преобразования целого текста в вектор, нам нужно получить средний вектор всех содержащихся в нём слов. Такой способ реализован в модели Doc2Vec в библиотеке gensim.

In [87]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tweets = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_df.text)]  # преобразуем наши тексты в объекты, понятные док-2-веку
d2v = Doc2Vec(tweets, min_count=2)                        # создадим модель Doc2Vec
d2v.train(tweets, total_examples=len(tweets), epochs=20)  # подберем веса коэффициентов внутри модели, которые больше будут подходить к нашему набору текстов

In [88]:
from sklearn.model_selection import train_test_split

# разобьем набор текстов на тренировочную и тестовую выборки
X_train_texts, X_test_texts, y_train, y_test = train_test_split(preprocessed_df.text, preprocessed_df.positive, test_size=0.2, random_state=21)

In [90]:
X_train_texts  # пока что наши тексты выглядят как списки слов в начальной форме, но нам нужно получить из этого векторы

114468    [спасибо, григорчуку, победу, черноморец, прос...
115015    [проверяли, сеня, больнички, хорошего, чувству...
147222          [тухнем, истории, павлуха, придет, сказала]
119060    [моему, ходили, водкой, часа, этим, обработать...
74753                                    [похоже, заметили]
                                ...                        
81968     [добавил, фотографий, альбом, дело, гуляли, св...
140036    [всем, буду, писать, пиздец, происходит, недел...
202552    [ооо, чувак, понимаю, жил, однокомнатной, квар...
70863     [родители, защищают, свое, говно, ахахаха, кла...
80841     [мелким, коллекция, фишек, всеми, покемонаме, ...
Name: text, Length: 181467, dtype: object

In [91]:
def transform_text_array_to_vector_dataframe(text_array):
    """Функция, которая преобразует одномерный колонку списков слов из текстов
    в датафрейм со значениями векторов этих текстов"""
    columns = [str(n) for n in range(d2v.vector_size)]               # задаем список названий колонок - просто порядковые номера
    vectors_ndarray = text_array.apply(d2v.infer_vector).to_list()  # прогоняем каждый текст через модель doc2vec и формируем многомерный массив чисел
    return pd.DataFrame(vectors_ndarray, columns=columns)            # оборачиваем его в датафрейм для удобства


X_train = transform_text_array_to_vector_dataframe(X_train_texts)    # наконец создадим датафреймы, которые сможем подать в модель классификации
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

Вы уже познакомились с некоторыми "деревянными" методами машинного обучения - решающим деревом и случайным лесом. Градиентный бустинг - это итеративный способ построения классификации, полученный (как и случайный лес) путем комбинации нескольких алгоритмов. Сначала строится обычное решающее дерево. Затем строится ряд моделей, предсказывающих ошибку исходной модели. Эти предсказания вычитаются из исходной модели. Таким образом, в итоге мы имеем один классификатор, но намного более точный, чем обычное решающее дерево. Чем больше итераций этого алгоритма будет проведено, тем выше получится качество модели, но она будет дольше обучаться.

Градиентный бустинг над решающими деревьями - это, пожалуй, самая распространенная на сегодняшний день модель машинного обучения. Ее используют для решениях многих задач, начиная кредитным скорингом и заканчивая товарным спросом и антифродом. Мы будем использовать эту модель для классификации наших текстов.

Модель градиентного бустинга есть в библиотеке sklearn, но на больших данных она будет обучаться долго. На рынке сейчас популярны несколько оптимизированных реализаций градиентного бустинга. Самые известные - xgboost, lightgbm и catboost. Рассмотрим xgboost, но вы можете использовать любую из этих трех.

In [72]:
# установим библиотеку
!pip install xgboost

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 665 kB 1.8 MB/s eta 0:00:01
^C
ERROR: Operation cancelled by user


Работать с моделью xgboost можно так же, как с моделями sklearn: fit и predict. Основные гиперпараметры - максимальная глубина деревьев модели и количество деревьев.

In [92]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

[19:57:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

          -1       0.61      0.61      0.61     22300
           1       0.63      0.62      0.62     23067

    accuracy                           0.62     45367
   macro avg       0.62      0.62      0.62     45367
weighted avg       0.62      0.62      0.62     45367



# Задание 2

В этом задании от вас требуется провести классификацию текстов с использованием градиентного бустинга. Постройте такую модель, которая даст наилучший результат по метрике precision к классу 0, подобрав гиперпараметры:
- минимальная встречаемость слова в текстах в doc2vec
- максимальная глубина деревеьев в бустинге
- количество деревьев в бустинге

Дайте ответ на вопрос: лучше использовать более глубокие или более мелкие деревья в модели градиентного бустинга?

In [94]:
# код выполнения задания 2

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tweets = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_df.text)]  # преобразуем наши тексты в объекты, понятные док-2-веку
d2v = Doc2Vec(tweets, min_count=2)                        # создадим модель Doc2Vec
d2v.train(tweets, total_examples=len(tweets), epochs=20)  # подберем веса ко

from sklearn.model_selection import train_test_split

# разобьем набор текстов на тренировочную и тестовую выборки
X_train_texts, X_test_texts, y_train, y_test = train_test_split(preprocessed_df.text, preprocessed_df.positive, test_size=0.2, random_state=21)
X_train_texts 
def transform_text_array_to_vector_dataframe(text_array):
    """Функция, которая преобразует одномерный колонку списков слов из текстов
    в датафрейм со значениями векторов этих текстов"""
    columns = [str(n) for n in range(d2v.vector_size)]               # задаем список названий колонок - просто порядковые номера
    vectors_ndarray = text_array.apply(d2v.infer_vector).to_list()  # прогоняем каждый текст через модель doc2vec и формируем многомерный массив чисел
    return pd.DataFrame(vectors_ndarray, columns=columns)            # оборачиваем его в датафрейм для удобства


X_train = transform_text_array_to_vector_dataframe(X_train_texts)    # наконец создадим датафреймы, которые сможем подать в модель классификации
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

# !pip install xgboost

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=2, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

xgb = XGBClassifier(max_depth=100, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

xgb = XGBClassifier(max_depth=10, n_estimators=10)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

xgb = XGBClassifier(max_depth=10, n_estimators=100)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

[22:18:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

          -1       0.59      0.59      0.59     22431
           1       0.60      0.60      0.60     22936

    accuracy                           0.60     45367
   macro avg       0.60      0.60      0.60     45367
weighted avg       0.60      0.60      0.60     45367

[22:19:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

          -1       0.61      0.61      0.61     22